In [3]:
import torch
from torch import nn
import torchcodec
import matplotlib.pyplot as plt

RuntimeError: Could not load libtorchcodec. Likely causes:
          1. FFmpeg is not properly installed in your environment. We support
             versions 4, 5, 6 and 7.
          2. The PyTorch version (2.8.0+cu126) is not compatible with
             this version of TorchCodec. Refer to the version compatibility
             table:
             https://github.com/pytorch/torchcodec?tab=readme-ov-file#installing-torchcodec.
          3. Another runtime dependency; see exceptions below.
        The following exceptions were raised as we tried to load libtorchcodec:
        
[start of libtorchcodec loading traceback]
FFmpeg version 7: libnppicc.so.12: cannot open shared object file: No such file or directory
FFmpeg version 6: libnppicc.so.12: cannot open shared object file: No such file or directory
FFmpeg version 5: libnppicc.so.12: cannot open shared object file: No such file or directory
FFmpeg version 4: libnppicc.so.12: cannot open shared object file: No such file or directory


In [2]:
if torch.cuda.is_available():
    device = "cuda" # Use NVIDIA GPU (if available)
elif torch.backends.mps.is_available():
    device = "mps" # Use Apple Silicon GPU (if available)
else:
    device = "cpu" # Default to CPU if no GPU is available

print(f"Currently using: {device}")

Currently using: cuda


In [9]:
waveform, sample_rate = torchaudio.load("genres/blues/blues.00000.wav")

/home/nai/miniconda3/envs/pytorch-music-genre-classifier/lib/python3.13/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


RuntimeError: Couldn't find appropriate backend to handle uri genres/blues/blues.00000.wav and format None.